# Overall Process and Specifications for Document-Processing to product novel Alpaca-Prompt formatted foundation model fine-tuning 'data-sets'

## Notes:
1. This preliminary code does not include the auto-formatting step, but instead focuses on the specificiations for such a system (which could be 'manual' as well)

2. This code for a google colab currently uses unsloth packages for a mistral model, but assuming you can get your tokenizer for whatevre model the overall process ~should work more generally. (where anything with a ~should in reality does not work...)

3. Industry and terminology is here as usual confusing and unclear. For example 'dataset' has a very specific meaning referring to a specifically formatted list of dictionaries (python datastructures or json files not yet loaded) that contains three dictionary keys. 'Dataset' is the proto-prompt material used to later make the actual prompts to make the actual corpus-of-texts to train with. This process must me be make more clear for more people to participate and meaningfully discuss basic workflow.


This is designed to accomodate fine tuning workflows such as Unsloths:
e.g.
- https://colab.research.google.com/drive/17d3U-CAIwzmbDRqbZ9NnpHxCkmXB6LZ0

- more at https://github.com/unslothai/unsloth

- note: you may need to authenticate with your huggingface token:


```
# go here for token: https://huggingface.co/settings/tokens

from huggingface_hub import login
login(token="your_access_token")
```

In [15]:
from google.colab import userdata
huggerface_token = userdata.get('huggerface_token')

In [16]:
from huggingface_hub import login
login(token=huggerface_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# for full model install (more than just tokenizer)

In [1]:
"""
code for full model install (more than just tokenizer)
"""
# %%capture
# # Installs Unsloth, Xformers (Flash Attention) and all other packages!
# !pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# # We have to check which Torch version for Xformers (2.3 -> 0.0.27)
# from torch import __version__; from packaging.version import Version as V
# xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
# !pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton

* We support Llama, Mistral, Phi-3, Gemma, Yi, DeepSeek, Qwen, TinyLlama, Vicuna, Open Hermes etc
* We support 16bit LoRA or 4bit QLoRA. Both 2x faster.
* `max_seq_length` can be set to anything, since we do automatic RoPE Scaling via [kaiokendev's](https://kaiokendev.github.io/til) method.
* [**NEW**] We make Phi-3 Medium / Mini **2x faster**! See our [Phi-3 Medium notebook](https://colab.research.google.com/drive/1hhdhBa1j_hsymiW9m-WzxQtgqTH_NHqi?usp=sharing)
* [**NEW**] We make Gemma-2 9b / 27b **2x faster**! See our [Gemma-2 9b notebook](https://colab.research.google.com/drive/1vIrqH5uYDQwsJ4-OO3DErvuv4pBgVwk4?usp=sharing)
* [**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [2]:
"""
code for full model install (more than just tokenizer)
"""
# from unsloth import FastLanguageModel
# import torch
# max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
# dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
# load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# # 4bit pre quantized models we support for 4x faster downloading + no OOMs.
# fourbit_models = [
#     "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
#     "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
#     "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
#     "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
#     "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
#     "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
#     "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
#     "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
#     "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
#     "unsloth/Phi-3-medium-4k-instruct",
#     "unsloth/gemma-2-9b-bnb-4bit",
#     "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
# ] # More models at https://huggingface.co/unsloth

# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name = "unsloth/Mistral-Nemo-Base-2407",
#     max_seq_length = max_seq_length,
#     dtype = dtype,
#     load_in_4bit = load_in_4bit,
#     # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
# )

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Mistral patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/165k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.31G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/177k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.26M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

# for just tokenizer:


In [ ]:
# https://huggingface.co/mistralai/Mistral-Nemo-Instruct-2407
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-Nemo-Instruct-2407")

In [19]:
import json
from datasets import load_dataset

"""
Custom Data Prep for "Alpaca Prompt" formatted training sets

Workflow steps:


Step 1: Setup
- authenticate with huggingface if needed
- set up a notebook or other environment
- Ensure tokenizer is defined: Make sure you have imported and defined the tokenizer before using tokenizer.eos_token.
- define the functions you need, namely: alpaca prompt formatting function

Step 2: Create your Engineered custom_data_set_list_of_dicts
 from one or more source documents

Step 3: Store Custom Data as Json these are proto-prompt dicts

step 4: list of dictionaires loaded to list of py-dicts

         with with three key value pairs:
            "instruction": "string value",
            "input": "string value",
            "output":"string value",

Step 5: Use (map) the hydration function
    to create  {'text': [list of prompt strings]}
    where each prompt string is a hydration of the 3-keyed dictionary
    'dataset' plus the EOS_TOKEN for that model.

{
    "text": [
        "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nidentify if a cat is alive\n\n### Input:\nthe cat is moving\n\n### Response:\nthe cat is alive<EOS_TOKEN>",
        "Below ...\n\n### Instruction:\nDescribe ...\n\n### Response:\n...<EOS_TOKEN>",
        # More formatted prompts+<EOS_TOKEN>s...
    ]
}

i.e.
A List of Formatted Strings: Each formatted prompt string,
in the list of formatted prompt string
{'text':[list of formatted prompt strings]}
is created by combining the
instruction, input, and output fields
with the ALPACA_PROMPT template and appending the EOS_TOKEN.


"""

################
# Step 1: Setup
################
"""
Setup:
- set up a notebook or other environment
- select and get your tokenizer
- define the functions you need, namely: alpaca prompt formatting function
"""
from google.colab import userdata
huggerface_token = userdata.get('huggerface_token')

from huggingface_hub import login
login(token=huggerface_token)

"""
Full model install, optional
"""
# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name = "unsloth/Mistral-Nemo-Base-2407",
#     max_seq_length = max_seq_length,
#     dtype = dtype,
#     load_in_4bit = load_in_4bit,
#     # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
# )

"""
just tokenizer
"""
# https://huggingface.co/mistralai/Mistral-Nemo-Instruct-2407
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-Nemo-Instruct-2407")

EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN


# function for formatting each prompt-string for the list of formatted strings:
# {'text':[list of formatted prompt strings]}
def formatting_alpaca_prompts_func(dict_unit_of_training_data):
    """
    Args:
        A dictionary that contains three keys: instruction, input, and output

    Returns:
        A dictionary containning a 'hydrated' text: { "text" : hydrated_text, }

    The formatting_prompts_func function expects a dictionary (or a list of dictionaries) where each dictionary contains three keys: instruction, input, and output.

    Example Input Format
    Each dictionary in the dataset should look like this:

    dict_unit_of_training_data = {
        "instruction": "identify if a cat is alive",
        "input": "the cat is moving",
        "output": "the cat is alive"
    }

    Note: because of the map-function-workflow
    the EOS_TOKEN must be a global constant...so much for best practice.

    """

    ALPACA_PROMPT = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

    ### Instruction:
    {}

    ### Input:
    {}

    ### Response:
    {}"""


    instructions = dict_unit_of_training_data["instruction"]
    inputs       = dict_unit_of_training_data["input"]
    outputs      = dict_unit_of_training_data["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = ALPACA_PROMPT.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }


###############################################################
# Step 2: Create your Engineered custom_data_set_list_of_dicts
# from one or more source documents
###############################################################
"""
example: https://raw.githubusercontent.com/tatsu-lab/stanford_alpaca/main/alpaca_data.json
custom_data_set_list_of_dicts = [
    {
        "instruction": "",
        "input": "",
        "output": ""
    },
    {
        "instruction": "",
        "input": "",
        "output": ""
    },
    # etc. more dictionaries...
]
"""
# option 1: create your dic-list manually here, probably tedious and error prone, not recommended
# option 2: use a process such as format-checked LLM-outout to convert
# original unstructed documents (or structure / semi-structued data) into
# a 'dataset' of pre-prompt 3-key dictionaries

# Put Your Completed Data Set here
custom_data_set_list_of_dicts = [
    {
        "instruction": "",
        "input": "",
        "output": ""
    },
    {
        "instruction": "",
        "input": "",
        "output": ""
    },
    # etc. more dictionaries...
]


####################################
# Step 3: Store Custom Data as Json
#  these are proto-prompt dicts
####################################
"""
You can also share this stored-data with other developers.
"""
with open("custom_data_set_list_of_dicts.json", "w") as f:
    json.dump(custom_data_set_list_of_dicts, f)


##########################################################################################################
# Step 4: Load Custom Data from json -> to py dict ->
# -> to hydrated "formated" strings -> to...list prompt strings?

# Load the JSON file into a dataset using the load_dataset function from the datasets library.
##########################################################################################################

dataset = load_dataset("json", data_files={"train": "custom_data_set_list_of_dicts.json"})


############################################################
# Step 5: Use (map) the hydration function
#  to create  {'text': [list of prompt strings]} where each
#  prompt string is a hydration of the 3-keyed dictionary
#  dataset' plus the EOS_TOKEN for that model.
############################################################

# Apply the formatting function
dataset = dataset.map(formatting_alpaca_prompts_func, batched = True,)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

## Inspect

In [20]:
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 2
    })
})

## Terse Version, Not Recommended

In [17]:
import json
from datasets import load_dataset

# Ensure tokenizer is defined
from transformers import AutoTokenizer

# https://huggingface.co/mistralai/Mistral-Nemo-Instruct-2407
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-Nemo-Instruct-2407")

EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN

# Function for formatting each prompt-string for the list of formatted strings:
# {'text':[list of formatted prompt strings]}
def formatting_alpaca_prompts_func(dict_unit_of_training_data):
    """
    Args:
        dict_unit_of_training_data (dict): A dictionary that contains three keys: instruction, input, and output.

    Returns:
        dict: A dictionary containing a 'hydrated' text: { "text" : hydrated_text, }

    The formatting_prompts_func function expects a dictionary (or a list of dictionaries) where each dictionary contains three keys: instruction, input, and output.

    Example Input Format:
    Each dictionary in the dataset should look like this:

    dict_unit_of_training_data = {
        "instruction": "identify if a cat is alive",
        "input": "the cat is moving",
        "output": "the cat is alive"
    }
    """

    ALPACA_PROMPT = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

    ### Instruction:
    {}

    ### Input:
    {}

    ### Response:
    {}"""

    instructions = dict_unit_of_training_data["instruction"]
    inputs = dict_unit_of_training_data["input"]
    outputs = dict_unit_of_training_data["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = ALPACA_PROMPT.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

###############################################################
# Step 1: Create your Engineered custom_data_set_list_of_dicts
# from one or more source documents
###############################################################
"""
example: https://raw.githubusercontent.com/tatsu-lab/stanford_alpaca/main/alpaca_data.json
custom_data_set_list_of_dicts = [
    {
        "instruction": "",
        "input": "",
        "output": ""
    },
    {
        "instruction": "",
        "input": "",
        "output": ""
    },
    # etc. more dictionaries...
]
"""
# Option 1: Create your dictionary list manually here, probably tedious and error prone, not recommended
# Option 2: Use a process such as format-checked LLM-output to convert original unstructured documents (or structured/semi-structured data) into a 'dataset' of pre-prompt 3-key dictionaries

# Put Your Completed Data Set here
custom_data_set_list_of_dicts = [
    {
        "instruction": "identify if a cat is alive",
        "input": "the cat is moving",
        "output": "the cat is alive"
    },
    {
        "instruction": "Describe the general characteristics of cats.",
        "input": "Cats are small carnivorous mammals.",
        "output": "Cats are known for their agility, sharp claws, and teeth. They are often kept as pets and are known for their independent nature."
    },
    # etc. more dictionaries...
]

####################################
# Step 2: Store Custom Data as Json
####################################
"""
You can also share this stored-data with other developers.
"""
with open("custom_data_set_list_of_dicts.json", "w") as f:
    json.dump(custom_data_set_list_of_dicts, f)

###########################
# Step 3: Load Custom Data
###########################

dataset = load_dataset("json", data_files={"train": "custom_data_set_list_of_dicts.json"})

###########################################
# Step 4: Use (map) the hydration function
###########################################

# Apply the formatting function
dataset = dataset.map(formatting_alpaca_prompts_func, batched=True)


tokenizer_config.json:   0%|          | 0.00/181k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.26M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 2
    })
})